Agents

Scratch -> LLM + functions inside its prompt
OpenAI Function calling -> using the official api to call functions for LLMs
Build simple agents with langchain

In [2]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import OpenAI
from IPython.display import Markdown
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

output = get_response("Create a simple task list of 3 desktop things I can do on the terminal.")
Markdown(output)

1. Check disk usage: Use the "du" command to display the disk usage of files and directories. For example, type "du -sh ~/" to see the total size of your home directory.

2. Rename files in bulk: Use the "mv" command to rename multiple files at once. For example, if you have a folder with multiple files named "file1.txt", "file2.txt", etc., you can rename them all to "new_file1.txt", "new_file2.txt", etc., with a single command like "mv file*.txt new_file*.txt".

3. Monitor system resources: Use the "top" command to monitor the usage of system resources like CPU and memory. It displays a live-updated list of processes and their resource usage. Simply type "top" in the terminal and press Enter to start monitoring. Use "q" to quit.

Ok cool, so here we have three ideas of actions to perform:

Creating directories
Listing files
Removing files
Let's transform them into functions that we could call just like in any type of Python-based application.

In [4]:
import subprocess

def create_directory(directory_name):
    # Execute shell command to create directory
    subprocess.run(["mkdir", directory_name])

def create_file():
    subprocess.run(["touch", "test.txt"])

def list_files():
    subprocess.run(["ls"])

In [5]:
create_directory("test")

mkdir: cannot create directory ‘test’: File exists


OK, cool! now Notice that, here we added single parameters to the functions: create_directory(), create_file(), and we did this so that we can actually do real things instead of just always creating the same folders over and over.

Now, how can we actually put it all together so that given a task, a model can:

Plan the task
Execute actions to complete the task
Know when to call a function

This is actually an interesting problem, let's understand why is that the case by trying to hack our way into putting all of these together:

In [6]:
 class ModelWithTools:
    def __init__(self, model):
        self.model = model
    
    def get_response(self, prompt_question):
        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                    {"role": "user", "content": prompt_question}]
        )
        
        return response.choices[0].message.content
    
    def create_directory(self, directory_name):
        subprocess.run(["mkdir", directory_name])
    
    def create_file(self, file_name):
        subprocess.run(["touch", file_name])
    
    def list_files(self):
        subprocess.run(["ls"])
    
    

model = ModelWithTools("gpt-3.5-turbo-16k")
task_description = "Create a folder called 'lucas-the-agent-master'. Inside that folder, create a file called 'the-10-master-rules.md"
output = model.get_response(f"""Given this task: {task_description}, \n
                            Consider you have access to the following functions:
                            
    def create_directory(self, directory_name):
        '''Function that creates a directory given a directory name.'''
        subprocess.run(["mkdir", directory_name])
    
    def create_file(self, file_name):
        '''Function that creates a file given a file name.'''
        subprocess.run(["touch", file_name])
    
    def list_files(self):
       '''Function that lists all files in the current directory.'''
        subprocess.run(["ls"])
    
    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    """)

Markdown(output)

create_directory('lucas-the-agent-master')

Hey! Look at that the output is that function! Now, all we need is to direct this output to be executed somehow!

In [7]:
exec("model." + output)

Yessss! We did it! All we had to do is to use the Python builtin method exec connected with the function call we got from the model's response! To avoid having to connect an outside function let's add some smart functionalities to our class to bind these capabilities all together.

In [8]:
class ModelWithTools:
    def __init__(self, model):
        self.model = model
    
    def get_response(self, prompt_question):
        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                    {"role": "user", "content": prompt_question}]
        )
        
        return response.choices[0].message.content
    
    def create_directory(self, directory_name):
        subprocess.run(["mkdir", directory_name])
    
    def create_file(self, file_name):
        subprocess.run(["touch", file_name])
    
    def list_files(self):
        subprocess.run(["ls"])
    
    def execute_function_call(self, function_call_string: str):
        exec(function_call_string)


model = ModelWithTools("gpt-3.5-turbo-16k")
task_description = "Create a folder called 'lucas-the-unoriginal-joker'."
output = model.get_response(f"""Given a task that will be fed as input, and consider you have access to the following functions:
                            
    def create_directory(self, directory_name):
        '''Function that creates a directory given a directory name.'''
        subprocess.run(["mkdir", directory_name])
    
    def create_file(self, file_name):
        '''Function that creates a file given a file name.'''
        subprocess.run(["touch", file_name])
    
    def list_files(self):
       '''Function that lists all files in the current directory.'''
        subprocess.run(["ls"])             
    
    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    For example:
    
    task: 'create a folder named lucas-the-agent-master'
    output: model.create_directory('lucas-the-agent-master')
    
    task: 'create a file named the-10-master-rules.md'
    output: model.create_file('the-10-master-rules.md')
    
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    task: {task_description}
    output:\n
    """)

Markdown(output)

model.create_directory('lucas-the-unoriginal-joker')

In [9]:
model.execute_function_call(output)

And there we have it! We connected our model to the tools!

This is great, but what if we wanted to perform multiple actions?

How about changing our prompt so that our output is a python list of function calls and then just looping over those lists and exeucting them one by one?

Let's try that:

In [10]:
model = ModelWithTools("gpt-3.5-turbo-16k")
task_description = "Create a folder called 'lucas-the-very-unoriginal-joker'. Inside that folder create a file called 'the-10-unoriginal-rules-of-comedy.md'."
output = model.get_response(f"""Given a task that will be fed as input, and consider you have access to the following functions:
                            
    def create_directory(self, directory_name):
        '''Function that creates a directory given a directory name.'''
        subprocess.run(["mkdir", directory_name])
    
    def create_file(self, file_name):
        '''Function that creates a file given a file name.'''
        subprocess.run(["touch", file_name])
    
    def list_files(self):
       '''Function that lists all files in the current directory.'''
        subprocess.run(["ls"])  
    .
    Your output should be the a list of function calls to be executed to complete the task containing the necessary arguments.
    For example:
    
    task: 'create a folder named test-dir'
    output_list: [model.create_directory('test-dir')]
    
    task: 'create a file named file.txt'
    output_list: [model.create_file('file.txt')]
    
    task: 'Create a folder named lucas-dir and inside that folder create a file named lucas-file.txt'
    output_list: [model.create_directory('lucas-dir'), model.create_file('lucas-dir/lucas-file.txt')]
    
    The OUTPUT SHOULD ONLY BE A PYTHON LIST WITH THE FUNCTION CALLS INSIDE and NOTHING ELSE.
    task: {task_description}
    output_list:\n
    """)

Markdown(output)

[model.create_directory('lucas-the-very-unoriginal-joker'), model.create_file('lucas-the-very-unoriginal-joker/the-10-unoriginal-rules-of-comedy.md')]

In [11]:
model.execute_function_call(output)

At this point we can start identifying a lot of issues with this approach despite our early sucess:

- Uncertainty of model's outputs can affect our ability to reliably call the functions
- We need more structured ways to prepare the inputs of the function calls
- We need better ways to put everything together (just feeding the entire functions like this makes it a very clunky and non-scalable framework for more complex cases)

There are many more issues but starting with these, we can now look at frameworks and see how they fix these issues and with that in mind understand what is behind their implementations!

I personally think this is a much better way to understand what is going on behind agents in practice rather than just use the more higher level frameworks right of the bat!

OpenAI Functions

Ok, let's first understand how [OpenAI](https://openai.com/) the company behind ChatGPT, allows for these function call implementations in its API.

OpenAI implemented a [function calling API](https://platform.openai.com/docs/guides/function-calling) which is a standard way to connect their models to outside tools like in the very simple example we did above.

According to their [official documentation](https://platform.openai.com/docs/guides/function-calling#:~:text=The%20basic%20sequence,to%20the%20user.) the sequence of steps for function calling is as follows:
1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

Below is an example taken from their official documentation:

In [12]:
from openai import OpenAI
import json

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
output = run_conversation()
output

ChatCompletion(id='chatcmpl-9QA1S5XLjsznv0NFQcNVqShqOxcLK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Currently, the weather in San Francisco is 72°C, in Tokyo it's 10°C, and in Paris it's 22°C.", role='assistant', function_call=None, tool_calls=None))], created=1716022082, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=29, prompt_tokens=169, total_tokens=198))

In [13]:
output.choices[0].message.content

"Currently, the weather in San Francisco is 72°C, in Tokyo it's 10°C, and in Paris it's 22°C."

In [14]:
import json

def create_directory(directory_name):
    """Function that creates a directory given a directory name."""""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})

In [15]:
create_directory("directory1")

'{"directory_name": "directory1"}'

In [17]:
tool_create_directory = {
    "type": "function",
    "function": {
        "name": "create_directory",
        "description": "Create a directory given a directory name.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create.",
                }
            },
            "required": ["directory_name"],
        },
    },
}

In [18]:
import json

def run_terminal_task():
    messages = [{"role": "user", "content": "Create a folder called 'lucas-the-super-unoriginal-joker'."}]
    tools = [tool_create_directory]  
    print("messages:", messages)
    print("tools:", tools)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    print("response_message:", response_message)
    tool_calls = response_message.tool_calls
    print("tool_calls:", tool_calls)
    # Step 2: check if the model wanted to call a function
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_directory": create_directory,
        }
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            print("function_name:", function_name)
            function_to_call = available_functions[function_name]
            print("function_to_call:", function_to_call)
            function_args = json.loads(tool_call.function.arguments)
            print("function_args:", function_args)
            function_response = function_to_call(
                directory_name=function_args.get("directory_name"),
            )
            print("function_response:", function_response)
            print("tool_call.id:", tool_call.id)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
            print("messages:", messages)
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
        )
        return second_response

output = run_terminal_task()
output

messages: [{'role': 'user', 'content': "Create a folder called 'lucas-the-super-unoriginal-joker'."}]
tools: [{'type': 'function', 'function': {'name': 'create_directory', 'description': 'Create a directory given a directory name.', 'parameters': {'type': 'object', 'properties': {'directory_name': {'type': 'string', 'description': 'The name of the directory to create.'}}, 'required': ['directory_name']}}}]
response_message: ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_baONgSa3FjfqHXjFN1IqQqte', function=Function(arguments='{\n  "directory_name": "lucas-the-super-unoriginal-joker"\n}', name='create_directory'), type='function')])
tool_calls: [ChatCompletionMessageToolCall(id='call_baONgSa3FjfqHXjFN1IqQqte', function=Function(arguments='{\n  "directory_name": "lucas-the-super-unoriginal-joker"\n}', name='create_directory'), type='function')]
function_name: create_directory
function_to_call: <function create_d

ChatCompletion(id='chatcmpl-9QANUBrNXPpBNgucgOj5ywX0XDYds', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I have created a folder called 'lucas-the-super-unoriginal-joker' for you.", role='assistant', function_call=None, tool_calls=None))], created=1716023448, model='gpt-3.5-turbo-16k-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=69, total_tokens=89))

In [19]:
output.choices[0].message.content

"I have created a folder called 'lucas-the-super-unoriginal-joker' for you."

Agents

In [22]:
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

@tool
def create_directory(directory_name):
    """Function that creates a directory given a directory name."""""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})

@tool
def create_file(file_path):
    """Function that creates a file given a file path."""""
    subprocess.run(["touch", file_path])
    return json.dumps({"file_path": file_path})

@tool
def some_other_function():
    """Function that does something else."""""
    return json.dumps({"some": "response"})    

tools = [create_directory, create_file]

llm_chat = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate.from_messages(
[
    ("system","You are very powerful assistant that helps\
                users perform tasks in the terminal."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm_chat.bind(functions=[format_tool_to_openai_function(t) for t in tools])

agent = (
{
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(
        x["intermediate_steps"]
    ),
}
| prompt
| llm_with_tools
| OpenAIFunctionsAgentOutputParser())

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

action_input = "Create a folder called 'lucas-the-random-joker2' and create a file inside this folder called 'the-10-random-rules-of-comedy.md'"

agent_executor.invoke({"input": action_input})

/home/ai/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/ai/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `create_directory` with `{'directory_name': 'lucas-the-random-joker2'}`


{"directory_name": "lucas-the-random-joker2"}
Invoking: `create_file` with `{'file_path': 'lucas-the-random-joker2/the-10-random-rules-of-comedy.md'}`


{"file_path": "lucas-the-random-joker2/the-10-random-rules-of-comedy.md"}Folder 'lucas-the-random-joker2' has been created and a file 'the-10-random-rules-of-comedy.md' has been created inside the folder.

> Finished chain.


{'input': "Create a folder called 'lucas-the-random-joker2' and create a file inside this folder called 'the-10-random-rules-of-comedy.md'",
 'output': "Folder 'lucas-the-random-joker2' has been created and a file 'the-10-random-rules-of-comedy.md' has been created inside the folder."}